### Exporting keys

In [11]:
import configparser
import os

In [5]:
config_file = 'keys.cfg'

In [10]:
config = configparser.ConfigParser()
config.read(config_file)

['keys.cfg']

In [18]:
os.environ['SPOTIPY_CLIENT_ID'] = config['SPOTIFY']['CLIENT_ID']
os.environ['SPOTIPY_CLIENT_SECRET'] = config['SPOTIFY']['CLIENT_SECRET']

### Spotify searching

In [20]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

spotipy = spotipy.Spotify(auth_manager=SpotifyClientCredentials())

In [24]:
results = spotipy.search(q='artist: BTS', type='artist')
item = results['artists']['items'][0]
item

{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Nrfpe0tUJi4K4DXYWgMUX'},
 'followers': {'href': None, 'total': 20986740},
 'genres': ['k-pop', 'k-pop boy group'],
 'href': 'https://api.spotify.com/v1/artists/3Nrfpe0tUJi4K4DXYWgMUX',
 'id': '3Nrfpe0tUJi4K4DXYWgMUX',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/83971573ae849bb366ff3d9d24623edd938805df',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/7fe2be9666e4f5cf4a96316086aaa92ba5b6376d',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ef0cba018c61d4e9e3225cee946be3e1a03be75f',
   'width': 160}],
 'name': 'BTS',
 'popularity': 93,
 'type': 'artist',
 'uri': 'spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX'}